# Импорт библиотек

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore")

# Загрузка данных

In [ ]:
# checkUID - id чека
# id_tt_cl - id торговой точки
# price_retail - базовая цена
# quantity - кол-во
# basePrice - цена позиции (после применения скидок)
# baseSum - сумма для позиции (после применения скидок)
# date_ch - дата
# time_ch - время
# order_type - тип чека

febr = pd.read_csv('2_febr.csv', sep=';')
march = pd.read_csv('2_march.csv', sep=';')
april = pd.read_csv('2_april.csv', sep=';')
may = pd.read_csv('2_may.csv', sep=';')
june = pd.read_csv('svao_june.csv', sep=';')

In [ ]:
# id_tt - id торговой точки
# adress - адрес
# hours - часы работы
# shirota - широта
# dolgota - долгота
# ploshad - площадь магазина
# city_tt - город 
# район - район
# shtat - штат
# qty_kassa - кол-во касс
# qty_kassa_si - кол-во касс самообслуживания
# format - формат магазина

post = pd.read_csv('2_ost.csv',  sep=';')
post['post'] = ost['post'].map(lambda x: x.replace(',', '.'))

# Корреляционный анализ

## 1) Продажи

Найдём корреляции объёмов продаж каждого из продаваемых товаров с выручкой

In [ ]:
# Соединим датасеты продаж в один общий датасет
months_concat = pd.concat([febr, march, may, april, may, june])
months_concat.head()

# Сгруппируем строки по столбцам "id торговой точки", "дата", "id товара" и посчитаем для каждой уникальной группы суммарный объём продаж
quantity = pd.DataFrame(months_concat.groupby(['id_tt_cl', 'date_ch', 'id_tov_cl'])['Quantity'].sum()).unstack().fillna(0)

# Сгруппируем строки по столбцам "id торговой точки", "дата" и посчитаем для каждой уникальной группы суммарную выручку
base_sum = pd.DataFrame(months_concat.groupby(['id_tt_cl', 'date_ch'])['BaseSum'].sum())
base_sum.head()

# Соединим полученные датасеты по полям "id торговой точки", "дата"
final_df1 = base_sum.merge(quantity, on=['id_tt_cl', 'date_ch'])

# Построим матрицу корреляций Спирмана 
corr = final_df1.corr(method='spearman')

# Найдём товары, продажи которых лучше/хуже всего коррелируют с выручкой
best_corr1 = pd.DataFrame(corr[corr > 0.6]['BaseSum']).sort_values(by='BaseSum', ascending=False).dropna()
worst_corr1 = pd.DataFrame(corr[corr < 0]['BaseSum']).sort_values(by='BaseSum', ascending=False).dropna()

## 2) Привозы

Найдём корреляции привозов каждого из товаров с выручкой

In [ ]:
# Заменим в датасете поставок полную дату на месяц
def set_month(x: str)->str:
    if x in pd.date_range(start='2022.03.01', end='2022.03.31'):
        return 'martch'
    elif x in pd.date_range(start='2022.04.01', end='2022.04.30'):
        return 'april'
    else:
        return 'may'

post['date_tt'] = post['date_tt'].map(set_month)

# Сгруппируем строки по столбцам "id торговой точки", "дата", "id товара" и посчитаем для каждой уникальной группы суммарный месячный привоз
monthly_post = pd.DataFrame(post.groupby(['id_tt', 'date_tt', 'id_tov']).sum()['morning']).unstack().fillna(0)

# Сгруппируем строки по столбцам "id торговой точки", "дата" и посчитаем для каждой уникальной группы суммарную месячную выручку
monthly_revenue = pd.DataFrame(post.groupby(['id_tt', 'date_tt']).sum()['morning']).unstack().fillna(0)

# Соединим полученные датасеты по полям "id торговой точки", "дата"
final_df2 = monthly_revenue.merge(monthly_post, on=['id_tt', 'date_tt']).fillna(0)

# Построим матрицу корреляций Спирмана 
corr = final_df2.corr(method='spearman')

# Найдём товары, привозы которых лучше/хуже всего коррелируют с выручкой
best_corr2 = pd.DataFrame(corr[corr > 0.6]['BaseSum']).sort_values(by='BaseSum', ascending=False).dropna()
worst_corr2 = pd.DataFrame(corr[corr < 0]['BaseSum']).sort_values(by='BaseSum', ascending=False).dropna()

# Формирование кластеров похожих точек

Итак, подход состоит в том, чтобы найти две похожие по определенным критериям торговые точки. Причем такие, чтобы у одной тренд выручки был положительный за определенный период времени, а у другой - отрицательный. Затем нужно сравнить их ассортимент с учетом проведенного выше анализа. В качестве теста изменить(при необходимости) ассортимент точки с отрицательным трендом; провести сравнение с точкой , которая тоже имеет отрицаельный тренд и похожа на две рассмотренных (т.е. все 3 входят в один кластер). Я выбрал три критерия близости, которые считаю довольно показательными

## 1) Суммарная выручка за все доступные месяцы

Будем считать точки похожими, если за 5 месяцев их суммарная выручка получилась примерно одинаковой 

In [ ]:
# Найдём для каждой торговой точки среднюю выручку за всё время
tt_mean_all_time = pd.DataFrame(months_concat\
                                .groupby('id_tt_cl')['BaseSum'].sum()).sort_values(by='BaseSum')

## 2) Доля выручки с онлайн заказов

Будем считать точки похожими, если доля выручки с онлайн продаж от общей суммы примерно совпадает

In [ ]:
tt_mean_all_time['offline_check'] = pd.DataFrame(months_concat\
                                                 [months_concat['order_type']=='offline']\
                                                 .groupby('id_tt_cl')['BaseSum']\
                                                 .sum()).sort_values(by='BaseSum')

tt_mean_all_time['online_check'] = pd.DataFrame(months_concat\
                                                 [months_concat['order_type']=='online']\
                                                 .groupby('id_tt_cl')['BaseSum']\
                                                 .sum()).sort_values(by='BaseSum')

tt_mean_all_time['part_of_online'] = tt_mean_all_time['online_check']/(tt_mean_all_time['offline_check'] + tt_mean_all_time['online_check'])

## 3) Приоритетные группы товаров

Будем считать точки похожими, если совпадает направленность ассортимента - например, если у одной точки наибольшая часть привозов приходится на молочную продукцию, а в другой её вообще не продают, то они никак не могут входить в один кластер

In [ ]:
# Отнормируем столбцы, т.к. объём разных товаров измеряется в разных шкалах
scaler = MinMaxScaler()
post['post'] = scaler.fit_transform(np.array(ost['post']).reshape(-1, 1))

# Надём ранжировки групп товаров для каждой торговой точки 
groups = pd.DataFrame(ost.groupby(by=['id_tt', 'id_group'])['post'].sum()).sort_values(by=['id_tt', 'post'], ascending=False)

# Выбор конкретных точек для исследования

Теперь нужно выбрать три точки из одного кластера так, чтобы они подошли нам для исследования с точки зрения описанного выше подхода (трендов выручки)

## 1) Графики изменения выручки по месяцам

In [ ]:
# choosed_tt_id - список с которые являются частью одного из кластеров (нет возможности показать его из-за NDA)
for x in choosed_tt_id:
    lst = []
    lst.append(febr[febr['id_tt_cl'] == x]['BaseSum'].sum())
    lst.append(march[march['id_tt_cl'] == x]['BaseSum'].sum())
    lst.append(april[april['id_tt_cl'] == x]['BaseSum'].sum())
    lst.append(may[may['id_tt_cl'] == x]['BaseSum'].sum())
    lst.append(june[june['id_tt_cl'] == x]['BaseSum'].sum())
    plt.scatter([1,2,3,4,5], lst);

## 2) Графики изменения выручки по дням

In [ ]:
for x in choosed_tt_id:
    lst = []
    lst.extend(febr[febr['id_tt_cl'] == 15123].groupby(by='date_ch')['BaseSum'].sum())
    lst.extend(march[march['id_tt_cl'] == 15123].groupby(by='date_ch')['BaseSum'].sum())
    lst.extend(april[april['id_tt_cl'] == 15123].groupby(by='date_ch')['BaseSum'].sum())
    lst.extend(may[may['id_tt_cl'] == 15123].groupby(by='date_ch')['BaseSum'].sum())
    lst.extend(june[june['id_tt_cl'] == 15123].groupby(by='date_ch')['BaseSum'].sum())
    plt.plot(range(0, len(lst)), lst);

# Выводы

На основе полученных графиков было выбрано три точки одного кластера - в одной выручка непрерывно росла в течение длительного периода времени, в двух других - падала. При сравнении хорошей и одной из плохих точек было замечено, что в плохой точке проседают продажи некоторых товаров с очень высокой корреляцией. Таким образом, решение состоит в том, чтобы подтянуть привоз этих товаров, ограничив привоз товаров с низкой корреляцией, но которые лучше продаются уже в плохой точке. В течение месяца сравнивать ситуацию (тренд выручки) этой точки со второй плохой, но для которой не будем делать никаких изменений. Если нововведения окажутся успешными, то можем распространить их на всю сеть